# PyCitySchools

In [1]:
# dependencies
import pandas as pd
import os

In [2]:
# set file paths
schoolsPath = os.path.join("Resources", "schools_complete.csv")
studentsPath = os.path.join("Resources", "students_complete.csv")

# read in csv files
schools = pd.read_csv(schoolsPath)
students = pd.read_csv(studentsPath)

## Schools Data

In [3]:
schools.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


## Students Data

In [4]:
students.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [5]:
# combine both files into one csv dataset
schoolsNstudents = pd.merge(students, schools, on="school_name", how="left")
#schoolsNstudents

## Combined Data

In [6]:
schoolsNstudents.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## Calculations

In [7]:
# calculate the total number of schools
total_schools = schools["school_name"].count()
#total_schools

In [8]:
# calculate the total number of students
total_students = students["student_name"].count()
#total_students

In [9]:
# calculate the total budget
total_budget = schools["budget"].sum()
#total_budget

In [10]:
# calculate the average math score
average_math = students["math_score"].mean()
#average_math

In [11]:
# calculate the average reading score
average_reading = students["reading_score"].mean()
#average_reading

In [12]:
# calculate the percentage of students with a passing math score (70 or greater)
math_greater_70 = students.loc[students["math_score"] >= 70, :]
math_passing_percent = (len(math_greater_70) / total_students)*100
#math_passing_percent

In [13]:
# calculate the percentage of students with a passing reading score (70 or greater)
reading_greater_70 = students.loc[students["reading_score"] >= 70, :]
reading_passing_percent = (len(reading_greater_70) / total_students)*100
#reading_passing_percent

In [14]:
# calculate the percentage of students who passed math and reading (% Overall Passing)
passing_math_reading = students.loc[(students["math_score"] >= 70) & (students["reading_score"] >= 70), :]
overall_passing_percent = (len(passing_math_reading) / total_students)*100
#overall_passing_percent

In [15]:
# Create a dataframe to hold the above results
district_summary = pd.DataFrame({"Total Schools": [total_schools],
                        "Total Students": [total_students],
                        "Total Budget": [total_budget],
                        "Average Math Score": [average_math],
                        "Average Reading Score": [average_reading],
                        "% Passing Math": [math_passing_percent],
                        "% Passing Reading": [reading_passing_percent],
                        "% Overall Passing": [overall_passing_percent],
                        })

In [16]:
# formatting for better read of district summary
district_summary["Total Students"] = district_summary["Total Students"].map('{:,.0f}'.format)
district_summary["Total Budget"] = district_summary["Total Budget"].map('${:,.2f}'.format)

## District Summary

In [17]:
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [18]:
# calculate budget per student by schools
schools["Per Student Budget"] = (schools["budget"] / schools["size"])
schools = schools.sort_values("school_name", ascending=True).reset_index(drop=True)
#schools["Per Student Budget"] = schools["Per Student Budget"].astype(float)
#schools

In [19]:
# calculate the average math scores by schools
math_score_per_school = students.groupby("school_name") ["math_score"].sum()
students_per_school = students.groupby("school_name")["math_score"].count()
average_math_score_df = math_score_per_school / students_per_school
#average_math_score_df

In [20]:
# calculate the average reading scores by schools
reading_score_per_school = students.groupby("school_name") ["reading_score"].sum()
average_reading_score_df = reading_score_per_school / students_per_school
#average_reading_score_df

In [21]:
# calculate the percentage of students with a passing math score (70 or greater) per school
math_greater_70_per_school = students.loc[students["math_score"] >= 70, :]
group_math_greater_70_per_school = math_greater_70_per_school.groupby("school_name") ["math_score"].count()
math_passing_pecent_per_school_df = (group_math_greater_70_per_school / students_per_school) * 100
#math_passing_pecent_per_school_df

In [22]:
# calculate the percentage of students with a passing reading score (70 or greater) per school
reading_greater_70_per_school = students.loc[students["reading_score"] >= 70, :]
group_reading_greater_70_per_school = reading_greater_70_per_school.groupby("school_name") ["reading_score"].count()
reading_passing_pecent_per_school_df = (group_reading_greater_70_per_school / students_per_school) * 100
#reading_passing_pecent_per_school_df

In [23]:
# calculate the percentage of students who passed math and reading (% Overall Passing) per school
passing_math_reading_per_school = students.loc[(students["math_score"] >= 70) & (students["reading_score"] >= 70), :]
group_passing_math_reading_per_school = passing_math_reading_per_school.groupby("school_name") ["school_name"].count()
overall_passing_percent_per_school_df = (group_passing_math_reading_per_school / students_per_school)*100
#overall_passing_percent_per_school_df

In [24]:
# Collect all require student marks df and place in a summary dataframe
student_marks_summary_df = pd.DataFrame({"Average Math Score": average_math_score_df,
                                         "Average Reading Score": average_reading_score_df,
                                         "% Passing Math": math_passing_pecent_per_school_df,
                                         "% Passing Reading": reading_passing_pecent_per_school_df,
                                         "% Overall Passing": overall_passing_percent_per_school_df
                                          }).reset_index()
#student_marks_summary_df

In [25]:
# merge the schools and the student marks summary for school summary output; along with viewable formats
school_summary = pd.merge(schools, student_marks_summary_df, on="school_name", how="left")

del school_summary["School ID"]

school_summary = school_summary.rename(columns={"school_name": "School Name", 
                                                "type": "School Type",
                                                "size": "Total Students",
                                                "budget": "Total School Budget"                                    
                                                })

school_summary["Total Students"] = school_summary["Total Students"].map('{:,.0f}'.format)
school_summary["Total School Budget"] = school_summary["Total School Budget"].map('${:,.2f}'.format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map('${:.2f}'.format)

## School Summary

In [47]:
school_summary.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455


In [54]:
# Top 5 Performing Schools (By % Overall Passing)
school_summary_sort = school_summary.sort_values("% Overall Passing", ascending=False).reset_index(drop=True)

## Top 5 Performing Schools (By % Overall Passing)

In [55]:
school_summary_sort.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
1,Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
2,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
3,Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
4,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


In [49]:
#Bottom 5 Performing Schools (By % Overall Passing)
school_summary_sort = school_summary.sort_values("% Overall Passing", ascending=True).reset_index(drop=True)

## Bottom 5 Performing Schools (By % Overall Passing) 

In [53]:
school_summary_sort.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
1,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
2,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
3,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
4,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [29]:
# Math Scores by Grade
# Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
grade9_math_scores = (students.loc[students["grade"] == "9th"]).groupby("school_name")["math_score"].mean()
grade10_math_scores = (students.loc[students["grade"] == "10th"]).groupby("school_name")["math_score"].mean()
grade11_math_scores = (students.loc[students["grade"] == "11th"]).groupby("school_name")["math_score"].mean()
grade12_math_scores = (students.loc[students["grade"] == "12th"]).groupby("school_name")["math_score"].mean()
#grade12_math_scores

In [51]:
# Collect all require student math marks df by grade by school and place in a summary dataframe
math_scores_per_grade_summary_df = pd.DataFrame({"Grade 9 Math Avg": grade9_math_scores,
                                         "Grade 10 Math Avg": grade10_math_scores,
                                         "Grade 11 Math Avg": grade11_math_scores,
                                         "Grade 12 Math Avg": grade12_math_scores
                                          }).reset_index()

## Math Scores by Grade by Schools

In [52]:
math_scores_per_grade_summary_df.head()

,school_name,Grade 9 Math Avg,Grade 10 Math Avg,Grade 11 Math Avg,Grade 12 Math Avg
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164


In [31]:
# Reading Scores by Grade
# Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
grade9_reading_scores = (students.loc[students["grade"] == "9th"]).groupby("school_name")["reading_score"].mean()
grade10_reading_scores = (students.loc[students["grade"] == "10th"]).groupby("school_name")["reading_score"].mean()
grade11_reading_scores = (students.loc[students["grade"] == "11th"]).groupby("school_name")["reading_score"].mean()
grade12_reading_scores = (students.loc[students["grade"] == "12th"]).groupby("school_name")["reading_score"].mean()
#grade12_reading_scores

In [57]:
# Collect all require student reading marks df by grade by school and place in a summary dataframe
reading_scores_per_grade_summary_df = pd.DataFrame({"Grade 9 Reading Avg": grade9_reading_scores,
                                         "Grade 10 Avg Reading Avg": grade10_reading_scores,
                                         "Grade 11 Avg Reading Avg": grade11_reading_scores,
                                         "Grade 12 Avg Reading Avg": grade12_reading_scores
                                          }).reset_index()

## Reading Scores by Grade by Schools

In [56]:
reading_scores_per_grade_summary_df.head()

,school_name,Grade 9 Reading Avg,Grade 10 Avg Reading Avg,Grade 11 Avg Reading Avg,Grade 12 Avg Reading Avg
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699


In [33]:
# build another merge school summary dataframe for binning with excepted unformatted dtypes
school_summary_binning = pd.merge(schools, student_marks_summary_df, on="school_name", how="left")
del school_summary_binning["School ID"]
school_summary_binning = school_summary_binning.rename(columns={"school_name": "School Name", 
                                                "type": "School Type",
                                                "size": "Total Students",
                                                "budget": "Total School Budget"                                    
                                                })
#school_summary_binning

In [34]:
# check range of values in column
print(school_summary["Per Student Budget"].min())
print(school_summary["Per Student Budget"].max())

$578.00
$655.00


In [35]:
# create bins and labels as bins for scores based on spending ranges
bins = [0, 580, 595, 610, 625, 640, 655]
group_labels = ["$0-$579", "$580-$594", "$595-$609", "$610-$624", "$625-$639", "$640-$656"]

In [36]:
# cut Per Student Budget and place into bins
pd.cut(school_summary_binning["Per Student Budget"], bins, labels=group_labels).head()

0    $625-$639
1    $580-$594
2    $625-$639
3    $640-$656
4    $610-$624
Name: Per Student Budget, dtype: category
Categories (6, object): [$0-$579 < $580-$594 < $595-$609 < $610-$624 < $625-$639 < $640-$656]

In [37]:
# Place the data series into a new column inside of the DataFrame
school_summary_binning["Spending Ranges"] = pd.cut(school_summary_binning["Per Student Budget"], bins, labels=group_labels)
#school_summary_binning

## Scores by School Spending

In [58]:
# Create a GroupBy object based upon "Spending Ranges"
school_summary_bin_group = school_summary_binning.groupby("Spending Ranges")

In [60]:
# Find how many rows fall into each bin
print(school_summary_bin_group["Average Math Score"].count())

Spending Ranges
$0-$579      1
$580-$594    3
$595-$609    2
$610-$624    1
$625-$639    4
$640-$656    4
Name: Average Math Score, dtype: int64


In [62]:
# Get the average of each column within the GroupBy object
spending_ranges = school_summary_bin_group[["Average Math Score", 
                      "Average Reading Score", 
                      "% Passing Math", 
                      "% Passing Reading", 
                      "% Overall Passing"]].mean()

## Scores by School Spending

In [63]:
spending_ranges

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges,,,,,
$0-$579,83.274201,83.989488,93.867718,96.539641,90.582567
$580-$594,83.515798,83.915256,93.324222,96.634622,90.298423
$595-$609,83.599686,83.885211,94.230858,95.900287,90.216324
$610-$624,83.351499,83.816757,93.392371,97.138965,90.599455
$625-$639,78.505315,81.696400,73.076824,85.050359,62.945755
$640-$656,77.023555,80.957446,66.701010,80.675217,53.717613


In [39]:
# check range of values in column
print(school_summary_binning["Total Students"].min())
print(school_summary_binning["Total Students"].max())

427
4976


In [40]:
# create bins and labels as bins for scores based on school size
bins = [0, 1000, 3000, 5000]
group_labels = ["Small (0-1000)", "Medium (1000-2500)", "Large (2500-5000)"]

In [41]:
# cut Total Students and place into bins
pd.cut(school_summary_binning["Total Students"], bins, labels=group_labels).head()

0     Large (2500-5000)
1    Medium (1000-2500)
2    Medium (1000-2500)
3    Medium (1000-2500)
4    Medium (1000-2500)
Name: Total Students, dtype: category
Categories (3, object): [Small (0-1000) < Medium (1000-2500) < Large (2500-5000)]

In [42]:
# Place the data series into a new column inside of the DataFrame
school_summary_binning["School Size Ranges"] = pd.cut(school_summary_binning["Total Students"], bins, labels=group_labels)
#school_summary_binning

In [ ]:
# Create a GroupBy object based upon "School Size Ranges"
school_summary_bin_group = school_summary_binning.groupby("School Size Ranges")

In [66]:
# Find how many rows fall into each bin
print(school_summary_bin_group["Average Math Score"].count())

School Size Ranges
Small (0-1000)        2
Medium (1000-2500)    9
Large (2500-5000)     4
Name: Average Math Score, dtype: int64


In [67]:
# Get the average of each column within the GroupBy object
school_size = school_summary_bin_group[["Average Math Score", 
                      "Average Reading Score", 
                      "% Passing Math", 
                      "% Passing Reading", 
                      "% Overall Passing"]].mean()

## Scores by School Size 

In [65]:
school_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size Ranges,,,,,
Small (0-1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2500),81.176821,82.933187,84.649798,91.316412,78.299832
Large (2500-5000),77.063340,80.919864,66.464293,81.059691,53.674303


In [69]:
# check for the different object values in column
print(school_summary_binning["School Type"].unique())

['District' 'Charter']


In [70]:
scores_by_school_type = pd.DataFrame({"Average Math Score": school_summary_binning.groupby("School Type") ["Average Math Score"].mean(), 
                                      "Average Reading Score": school_summary_binning.groupby("School Type") ["Average Reading Score"].mean(),
                                      "% Passing Math": pd.to_numeric(school_summary_binning["% Passing Math"]).groupby(school_summary_binning["School Type"]).mean(),
                                      "% Passing Reading": pd.to_numeric(school_summary_binning["% Passing Reading"]).groupby(school_summary_binning["School Type"]).mean(),
                                      "% Overall Passing": pd.to_numeric(school_summary_binning["% Overall Passing"]).groupby(school_summary_binning["School Type"]).mean()
                                   })

## Scores by School Types

In [72]:
scores_by_school_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
